In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nest_asyncio

nest_asyncio.apply()

# Imports

In [ ]:
import asyncio
import operator

from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langgraph.types import Send
from langsmith import traceable
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Optional, TypedDict

load_dotenv()

# Vanilla workflow

In [ ]:
class Section(BaseModel):
    name: str = Field(description="The name of the section")
    description: str = Field(description="The description of the section")


class CompletedSection(BaseModel):
    name: str = Field(description="The name of the section")
    content: str = Field(description="The content of the section")


class Sections(BaseModel):
    sections: list[Section] = Field(description="The sections of the article")


class OrchestratorState(BaseModel):
    topic: str
    sections: Optional[list[Section]] = None
    completed_sections: Optional[list[CompletedSection]] = None
    final_report: Optional[str] = None


model = ChatOpenAI(model="gpt-4.1-mini")


@traceable
async def plan_sections(state: OrchestratorState):
    model_planner = model.with_structured_output(Sections)
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, you will generate the sections for a short article."
        ),
        HumanMessage(
            content=f"Generate the sections of an article about {state.topic}"
        ),
    ]
    response = await model_planner.ainvoke(messages)
    return response.sections


@traceable
async def write_section(section: Section) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic and a table of contents, you will generate the content of the article."
        ),
        HumanMessage(
            content=f"Generate the content of an article about {section.name} with the following description: {section.description}"
        ),
    ]
    response = await model.ainvoke(messages)
    return CompletedSection(name=section.name, content=response.content)


@traceable
def synthesizer(state: OrchestratorState) -> str:
    ordered_sections = state.completed_sections
    completed_sections_str = "\n\n".join(
        [section.content for section in ordered_sections]
    )
    return completed_sections_str


@traceable
async def run_workflow(topic: str) -> OrchestratorState:
    state = OrchestratorState(topic=topic)
    state.sections = await plan_sections(state)
    tasks = [write_section(section) for section in state.sections]
    state.completed_sections = await asyncio.gather(*tasks)
    state.final_report = synthesizer(state)
    return state


state = await run_workflow("Substance abuse of athletes")

# LangGraph implementation

In [ ]:
class Section(BaseModel):
    name: str = Field(description="The name of the section")
    description: str = Field(description="The description of the section")


class CompletedSection(BaseModel):
    name: str = Field(description="The name of the section")
    content: str = Field(description="The content of the section")


class Sections(BaseModel):
    sections: list[Section] = Field(description="The sections of the article")


class OrchestratorState(TypedDict):
    topic: str
    sections: list[Section]
    completed_sections: Annotated[list[CompletedSection], operator.add]
    final_report: str


class WorkerState(TypedDict):
    section: str
    completed_sections: Annotated[list[Section], operator.add]


model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)


def orchestrator(state: OrchestratorState) -> dict:
    model_planner = model.with_structured_output(Sections)
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic, you will generate the sections for a short article."
        ),
        HumanMessage(
            content=f"Generate the sections of an article about {state['topic']}"
        ),
    ]
    return {"sections": model_planner.invoke(messages).sections}


def write_section(state: WorkerState) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer specialized in SEO. Provided with a topic and a table of contents, you will generate the content of the article."
        ),
        HumanMessage(
            content=f"Generate the content of an article about {state['section'].name} with the following description: {state['section'].description}"
        ),
    ]
    section = CompletedSection(
        name=state["section"].name, content=model.invoke(messages).content
    )
    return {"completed_sections": [section]}


def synthesizer(state: OrchestratorState) -> str:
    ordered_sections = state["completed_sections"]
    completed_sections_str = "\n\n".join(
        [section.content for section in ordered_sections]
    )
    return {"final_report": completed_sections_str}


def assign_workers(state: OrchestratorState) -> dict:
    return [
        Send("write_section", {"section": section}) for section in state["sections"]
    ]


workflow = StateGraph(OrchestratorState)

workflow.add_node("orchestrator", orchestrator)
workflow.add_node("write_section", write_section)
workflow.add_node("synthesizer", synthesizer)

workflow.add_edge(START, "orchestrator")
workflow.add_conditional_edges("orchestrator", assign_workers, ["write_section"])
workflow.add_edge("write_section", "synthesizer")
workflow.add_edge("synthesizer", END)

chain = workflow.compile()

display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
state = chain.invoke({"topic": "Artificial Intelligence"})

In [ ]:
for section in state["sections"]:
    print(section.name)